In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torch torchvision

In [ ]:
import os
import shutil
import random
from pycocotools.coco import COCO
from pycocotools.cocoeval import COCOeval
import numpy as np
import skimage.io as io
import pylab
import pandas as pd
import json
import matplotlib.pyplot as plt
import glob

In [ ]:
# Clone github repo.
!git clone https://github.com/ultralytics/yolov5.git

# Change dir to yolov5
%cd yolov5/

# Install requirements
!pip install -r requirements.txt

import torch
from yolov5 import utils
display = utils.notebook_init()

In [ ]:
!wget https://github.com/ultralytics/yolov5/releases/download/v6.0/yolov5m.pt

In [ ]:
# Define the dataset.yaml file keys.
import yaml
parsed_yaml_file = dict()
parsed_yaml_file['path'] = ''
parsed_yaml_file['train'] = '/kaggle/input/ship-detection-sar/HRSID_YOLO/train'
parsed_yaml_file['val'] = '/kaggle/input/ship-detection-sar/HRSID_YOLO/val'
parsed_yaml_file['test'] = '/kaggle/input/ship-detection-sar/HRSID_YOLO/test'
parsed_yaml_file['nc'] = 1
parsed_yaml_file['names'] = ['Ship']
parsed_yaml_file

In [ ]:
# Overwrite the original .yaml file.
path_4_saving = '/kaggle/working/dataset.yaml'
with open(path_4_saving, 'w') as outfile:
    yaml.dump(parsed_yaml_file, outfile, default_flow_style=False)

In [ ]:
# Check that the .yaml file saved correctly.
a_yaml_file = open(path_4_saving)
parsed_yaml_file = yaml.load(a_yaml_file, Loader=yaml.FullLoader)
parsed_yaml_file

In [ ]:
!pip install wandb

In [ ]:
import wandb
personal_wandb_api_key = '182fb77b09d214e4546d4ff2422c05f1b8398b6f'
wandb.login(key=personal_wandb_api_key)

In [ ]:
!wandb login --relogin 182fb77b09d214e4546d4ff2422c05f1b8398b6f

In [ ]:
# !python train.py --img 1344 --batch 16 --epochs 20 --data $path_4_saving --weights /content/yolov5/yolov5m.pt

In [ ]:
!pip install torch

In [ ]:
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

In [ ]:
!pip install google.colab

In [ ]:
# from IPython.display import display
# from google.colab import files

# uploaded = files.upload()

In [ ]:
# Download model's weights.
!zip -r /content/exp2.zip './runs/train/exp'
# from google.colab import files
# files.download('/content/exp.zip')

In [ ]:
!python val.py --weights /kaggle/input/temporary2/best2.pt --data /kaggle/working/dataset.yaml --img 1344 --iou 0.5 --task test --save-json

In [ ]:
test_path = '/kaggle/input/ship-detection-sar/HRSID_YOLO/test/images'
weight_path = '/kaggle/input/temporary3/last.pt'

In [ ]:
# Inference on all test images.
!python detect.py --source $test_path --weights $weight_path --img 1344 --save-txt --save-conf --line-thickness 1

In [ ]:
# Zip the output results
folder_name = 'inference_on_imgs.zip'
!zip -r $folder_name /kaggle/working/yolov5/yolov5/runs/detect/exp

# Download the results
from google.colab import files
files.download(folder_name)

In [ ]:
# Zip the output results
import shutil

folder_name = 'inference_on_imgs.zip'
output_folder = '/kaggle/working/yolov5/runs/detect/exp'

# Create the zip file
shutil.make_archive('inference_on_imgs', 'zip', output_folder)

# The zip file will be available in the '/kaggle/working/' directory for download
print(f"Results zipped and saved as {folder_name} in the /kaggle/working/ directory.")


In [ ]:
# Inference on one random test image:
%cd /kaggle/working/yolov5/yolov5
f_name = random.choice(os.listdir(test_path))
path_2_image = os.path.join(test_path, f_name)
!python detect.py --weights $weight_path --img 640 --conf 0.25 --source $path_2_image

In [ ]:
# Plot the above image with it's predictions:
img = plt.imread(glob.glob(os.path.join('runs/detect/*', f_name))[0])
plt.figure(figsize=(10,10));
plt.imshow(img);

In [ ]:
!python /kaggle/working/yolov5/detect.py --source /kaggle/input/temporary5 --weights /kaggle/input/temporary3/last.pt --conf 0.5 --imgsz 1300 --line-thickness 1

In [ ]:
import zipfile
import requests
import cv2
import matplotlib.pyplot as plt
import glob 
import random
import os

In [ ]:
# Function to convert bounding boxes in YOLO format to xmin, ymin, xmax, ymax.
def yolo2bbox(bboxes):
    xmin, ymin = bboxes[0]-bboxes[2]/2, bboxes[1]-bboxes[3]/2
    xmax, ymax = bboxes[0]+bboxes[2]/2, bboxes[1]+bboxes[3]/2
    return xmin, ymin, xmax, ymax

In [ ]:
# Path to your dataset directory containing images
dataset_dir = "C:\\Users\\thatw\\\\akash\\Suhora Project\\HRSID_jpg\\HRSID_JPG\\JPEGImages"

# Path to your annotation file
annotation_file = "C:\\Users\\thatw\\akash\\Suhora Project\\HRSID_jpg\\HRSID_JPG\\annotations\\train2017.json"  # Assuming annotations folder is located inside the dataset directory
annotation_path = os.path.join(dataset_dir, "..", annotation_file)

# Load annotation data
with open(annotation_path, 'r') as f:
    annotation_data = json.load(f)

# Randomly select 5 images
random_images = random.sample(annotation_data['images'], 5)

# Create subplots
fig, axes = plt.subplots(1, 5, figsize=(15, 3))

# Loop through each randomly selected image
for ax, image_info in zip(axes, random_images):
    img_file = image_info['file_name']
    img_id = image_info['id']

    img_path = os.path.join(dataset_dir, img_file)

    # Load the image
    image = cv2.imread(img_path)

    # Find annotations for this image
    annotations = [ann for ann in annotation_data['annotations'] if ann['image_id'] == img_id]

    # Process annotations and draw bounding boxes on the image
    for annotation in annotations:
        bbox = annotation['bbox']
        x, y, w, h = bbox
        cv2.rectangle(image, (int(x), int(y)), (int(x + w), int(y + h)), (0, 255, 0), 2)

    # Convert the image from BGR to RGB (matplotlib uses RGB)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    # Display the image with annotations
    ax.imshow(image_rgb)
    ax.axis('on')

plt.subplots_adjust(wspace=0.05)
plt.tight_layout()
plt.show()

print("Mapping complete.")